In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

In [ ]:
!python --version

Python 3.10.11


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/thesis/train/cv_scaled.csv', sep=',')


In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/thesis/val/val-cv_out.csv', sep=',')

In [ ]:
df_test.head(5)

,0,1,2,3,4,5,6,7,8,9,...,508,509,510,511,512,513,514,515,516,517
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007299,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007299,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007299,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007299,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.007299,0.0


In [ ]:
df_test.shape

(102000, 518)

In [ ]:
df.shape[1]

518

REDUCTION FOR CV

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Prepare your data

# Define the dimensions of your input data
input_dim = df.shape[1]

# Define the architecture of the autoencoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Create the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mse', metrics = 'acc')

# Train the autoencoder
history = autoencoder.fit(df, df, epochs=10, batch_size=64, validation_split=0.15)

# Extract the encoder part
encoder = Model(input_layer, encoded)

# Apply the encoder for dimensionality reduction
reduced_data = encoder.predict(df)


Epoch 1/10
13561/13561 [==============================] - 58s 4ms/step - loss: 0.0022 - acc: 0.1631 - val_loss: 0.0021 - val_acc: 0.2000
Epoch 2/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0017 - acc: 0.1884 - val_loss: 0.0021 - val_acc: 0.1851
Epoch 3/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0017 - acc: 0.0278 - val_loss: 0.0021 - val_acc: 0.0016
Epoch 4/10
13561/13561 [==============================] - 48s 4ms/step - loss: 0.0017 - acc: 0.0039 - val_loss: 0.0021 - val_acc: 7.9659e-04
Epoch 5/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0017 - acc: 0.0011 - val_loss: 0.0021 - val_acc: 7.9659e-04
Epoch 6/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0017 - acc: 0.0011 - val_loss: 0.0021 - val_acc: 7.9659e-04
Epoch 7/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0017 - acc: 0.0089 - val_loss: 0.0021 - val_acc: 7.9659e-04
Epoch 8/10
13561/13561 [=

In [ ]:
X = pd.DataFrame(reduced_data)

In [ ]:
outlier_data = encoder.predict(df_test)

3188/3188 [==============================] - 4s 1ms/step


In [ ]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 518)]             0         
                                                                 
 dense (Dense)               (None, 128)               66432     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 64)                2112      
                                                                 
 dense_4 (Dense)             (None, 128)               8320      
                                                                 
 dense_5 (Dense)             (None, 518)               66822 

In [ ]:
import sklearn
print(sklearn.__version__)

1.2.2


In [ ]:
pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184097 sha256=4461a3f1de316ccb980107609bea1e7ee6965a5f2ff5f2be124db68b1f52a2e5
  Stored in directory: /root/.cache/pip/wheels/83/55/6b/552e083cf5509c0afe808b76cf434f1be284d01a112623bd37
Successfully built pyod


In [ ]:
import keras

In [ ]:
print(keras.__version__)

2.12.0


In [ ]:
import numpy
print(numpy.__version__)

1.22.4


In [ ]:
import pandas
print(pandas.__version__)

1.5.3


In [ ]:
import matplotlib
print(matplotlib.__version__)

3.7.1


In [ ]:
import re
print(re.__version__)

2.2.1


In [ ]:
import tensorflow

In [ ]:
print(tensorflow.__version__)

2.12.0


In [ ]:
X_out = pd.DataFrame(outlier_data)

In [ ]:
X.to_csv('/content/drive/MyDrive/thesis/train/reduce_ae_cv.csv', index=False)

NameError: ignored

In [ ]:
X_out.to_csv('/content/drive/MyDrive/thesis/val/val-x_out_oh.csv', index=False)

In [ ]:
X.shape

(1021009, 32)

In [ ]:
X_out.shape

(21006, 32)

TF-idf reduction

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/thesis/train/tf-idf_scaled.csv', sep=',')


In [ ]:
df_out = pd.read_csv('/content/drive/MyDrive/thesis/val/val-tf-idf_scaled.csv', sep = ',')

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Prepare your data

# Define the dimensions of your input data
input_dim = df.shape[1]

# Define the architecture of the autoencoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Create the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics = 'acc')

# Train the autoencoder
history = autoencoder.fit(df, df, epochs=10, batch_size=1000, validation_split=0.15)

# Extract the encoder part
encoder = Model(input_layer, encoded)

# Apply the encoder for dimensionality reduction
reduced_data = encoder.predict(df)

Epoch 1/10
868/868 [==============================] - 15s 9ms/step - loss: 0.0188 - acc: 0.1444 - val_loss: 0.0055 - val_acc: 0.2080
Epoch 2/10
868/868 [==============================] - 6s 7ms/step - loss: 0.0017 - acc: 0.5032 - val_loss: 0.0064 - val_acc: 0.2681
Epoch 3/10
868/868 [==============================] - 6s 7ms/step - loss: 0.0016 - acc: 0.5905 - val_loss: 0.0067 - val_acc: 0.9658
Epoch 4/10
868/868 [==============================] - 6s 7ms/step - loss: 0.0016 - acc: 0.9770 - val_loss: 0.0068 - val_acc: 0.9664
Epoch 5/10
868/868 [==============================] - 6s 7ms/step - loss: 0.0016 - acc: 0.9727 - val_loss: 0.0068 - val_acc: 0.9206
Epoch 6/10
868/868 [==============================] - 6s 7ms/step - loss: 0.0016 - acc: 0.9399 - val_loss: 0.0067 - val_acc: 0.9209
Epoch 7/10
868/868 [==============================] - 6s 7ms/step - loss: 0.0016 - acc: 0.9382 - val_loss: 0.0065 - val_acc: 0.9208
Epoch 8/10
868/868 [==============================] - 6s 7ms/step - loss: 0

In [ ]:
X = pd.DataFrame(reduced_data)

In [ ]:
X.to_csv('/content/drive/MyDrive/thesis/train/reduce_ae_tf-idf.csv', index=False)

In [ ]:
outlier = encoder.predict(df_out)

3188/3188 [==============================] - 4s 1ms/step


In [ ]:
x_out = pd.DataFrame(outlier)

In [ ]:
x_out.to_csv('/content/drive/MyDrive/thesis/val/val-reduce_ae_tf-idf.csv', index=False)

OH_DATA

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/thesis/train/oh_data.csv', sep=',')
x_out = pd.read_csv('/content/drive/MyDrive/thesis/val/val-x_out_oh.csv', sep=',')

In [ ]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# Prepare your data

# Define the dimensions of your input data
input_dim = df.shape[1]

# Define the architecture of the autoencoder
input_layer = Input(shape=(input_dim,))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# Create the autoencoder model
autoencoder = Model(input_layer, decoded)

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='mse', metrics = 'acc')

# Train the autoencoder
history = autoencoder.fit(df, df, epochs=10, batch_size=64, validation_split=0.15)

# Extract the encoder part
encoder = Model(input_layer, encoded)

# Apply the encoder for dimensionality reduction
reduced_data = encoder.predict(df)


Epoch 1/10
13561/13561 [==============================] - 54s 4ms/step - loss: 0.0022 - acc: 0.6216 - val_loss: 0.0023 - val_acc: 0.6973
Epoch 2/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0018 - acc: 0.8692 - val_loss: 0.0023 - val_acc: 0.8823
Epoch 3/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0018 - acc: 0.9370 - val_loss: 0.0023 - val_acc: 0.9281
Epoch 4/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0018 - acc: 0.9383 - val_loss: 0.0023 - val_acc: 0.9281
Epoch 5/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0018 - acc: 0.9383 - val_loss: 0.0023 - val_acc: 0.9281
Epoch 6/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0018 - acc: 0.9383 - val_loss: 0.0023 - val_acc: 0.9281
Epoch 7/10
13561/13561 [==============================] - 49s 4ms/step - loss: 0.0018 - acc: 0.9383 - val_loss: 0.0023 - val_acc: 0.9281
Epoch 8/10
13561/13561 [=================

In [ ]:
outlier_data = encoder.predict(x_out)

3188/3188 [==============================] - 4s 1ms/step


In [ ]:
df_oh = pd.DataFrame(reduced_data)
x_out_oh = pd.DataFrame(outlier_data)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mm_scaler = MinMaxScaler()
x_scaled_oh = mm_scaler.fit_transform(df_oh)

In [ ]:
x_out_oh = mm_scaler.transform(x_out_oh)

In [ ]:
train = pd.DataFrame(x_scaled_oh)
test = pd.DataFrame(x_out_oh)

In [ ]:
train.to_csv('/content/drive/MyDrive/thesis/train/reduce_ae_oh_data.csv', index=False)
test.to_csv('/content/drive/MyDrive/thesis/val/val-reduce_ae_oh_data.csv', index=False)